第02课-手搓一个土得掉渣的Agent.ipynb

In [2]:
import os
from dotenv import load_dotenv

# 加载环境变量
load_dotenv()
# 从环境变量中读取 DashScope（Qwen）API Key
api_key = os.getenv('DASHSCOPE_API_KEY')
base_url = "https://dashscope.aliyuncs.com/compatible-mode/v1"
chat_model = "qwen-plus"

print(api_key,base_url,chat_model)

sk-6a28680f8c7e4174b41ce71e235de4c8 https://dashscope.aliyuncs.com/compatible-mode/v1 qwen-plus


In [5]:
from openai import OpenAI
client = OpenAI(
    api_key = api_key,
    base_url = base_url
)

In [6]:
def get_completion(prompt):
    response = client.chat.completions.create(
        model = chat_model,
        messages = [{"role": "user", "content": prompt}],
    )
    return response.choices[0].message.content

In [7]:

response = get_completion("你是谁？")
print(response)

我是通义千问，阿里巴巴集团旗下的通义实验室自主研发的超大规模语言模型。我可以帮助你回答问题、创作文字，比如写故事、写公文、写邮件、写剧本、逻辑推理、编程等等，还能表达观点，玩游戏等。如果你有任何问题或需要帮助，欢迎随时告诉我！


In [8]:
sys_prompt = """你是一个聪明的客服。您将能够根据用户的问题将不同的任务分配给不同的人。您有以下业务线：
1.用户注册。如果用户想要执行这样的操作，您应该发送一个带有"registered workers"的特殊令牌。并告诉用户您正在调用它。
2.用户数据查询。如果用户想要执行这样的操作，您应该发送一个带有"query workers"的特殊令牌。并告诉用户您正在调用它。
3.删除用户数据。如果用户想执行这种类型的操作，您应该发送一个带有"delete workers"的特殊令牌。并告诉用户您正在调用它。
"""
registered_prompt = """
您的任务是根据用户信息存储数据。您需要从用户那里获得以下信息：
1.用户名、性别、年龄
2.用户设置的密码
3.用户的电子邮件地址
如果用户没有提供此信息，您需要提示用户提供。如果用户提供了此信息，则需要将此信息存储在数据库中，并告诉用户注册成功。
存储方法是使用SQL语句。您可以使用SQL编写插入语句，并且需要生成用户ID并将其返回给用户。
如果用户没有新问题，您应该回复带有 "customer service" 的特殊令牌，以结束任务。
"""
query_prompt = """
您的任务是查询用户信息。您需要从用户那里获得以下信息：
1.用户ID
2.用户设置的密码
如果用户没有提供此信息，则需要提示用户提供。如果用户提供了此信息，那么需要查询数据库。如果用户ID和密码匹配，则需要返回用户的信息。
如果用户没有新问题，您应该回复带有 "customer service" 的特殊令牌，以结束任务。
"""
delete_prompt = """
您的任务是删除用户信息。您需要从用户那里获得以下信息：
1.用户ID
2.用户设置的密码
3.用户的电子邮件地址
如果用户没有提供此信息，则需要提示用户提供该信息。
如果用户提供了这些信息，则需要查询数据库。如果用户ID和密码匹配，您需要通知用户验证码已发送到他们的电子邮件，需要进行验证。
如果用户没有新问题，您应该回复带有 "customer service" 的特殊令牌，以结束任务。
"""

In [12]:
class SmartAssistant:
    def __init__(self):
        self.sys_prompt = sys_prompt
        self.registered_prompt = registered_prompt
        self.query_prompt = query_prompt
        self.delete_prompt = delete_prompt

        self.client = client

        # Using a dictionary to store different sets of messages
        self.messages = {
            "system": [{"role": "system", "content": self.sys_prompt}],
            "registered": [{"role": "system", "content": self.registered_prompt}],
            "query": [{"role": "system", "content": self.query_prompt}],
            "delete": [{"role": "system", "content": self.delete_prompt}]
        }
        # Current assignment for handling messages
        self.current_assignment = "system"
    
    def get_response(self, user_input):
        self.messages[self.current_assignment].append({"role": "user", "content": user_input})
        while True:
            response = self.client.chat.completions.create(
                model = chat_model,
                messages= self.messages[self.current_assignment],
                temperature = 0.9,
                stream = False,
                max_tokens =2000)
            
            ai_response = response.choices[0].message.content

            if "registered workers" in ai_response:
                self.current_assignment = "registered"
                print("意图识别:",ai_response)
                print("switch to <registered>")
                self.messages[self.current_assignment].append({"role": "user", "content": ai_response})
            if "query workers" in ai_response:
                self.current_assignment = "query"
                print("意图识别:",ai_response)
                print("switch to <query>")
                self.messages[self.current_assignment].append({"role": "user", "content": ai_response})
            if "delete workers" in ai_response:
                self.current_assignment = "delete"
                print("意图识别:",ai_response)
                print("switch to <delete>")
            elif "customer service" in ai_response:
                print("意图识别:",ai_response)
                print("switch to <customer service>")
                self.messages["system"] += self.messages[self.current_assignment]
                self.current_assignment = "system"
                return ai_response
            else:
                self.messages[self.current_assignment].append({"role": "assistant", "content": ai_response})
                return ai_response
    def start_conversation(self):

        while True:
            user_input = input("User: ")
            if user_input.lower() in ['exit', 'quit']:
                print("Exiting conversation.")
                break
            response = self.get_response(user_input)
            print("Assistant:", response)


In [14]:
ai = SmartAssistant()
ai.start_conversation()

Assistant: 您好，我是一个AI助手，不太明白您说的“我是单欠打”是什么意思。您可以提供更多的信息给我吗？这样我就能更好地帮助您了。
Assistant: 我理解您可能在表达一些情绪，但请使用适当的语言进行沟通。如果您有任何问题或需要帮助，请告诉我，我会尽力提供支持。同时，请确保我们的交流保持尊重和礼貌。如果您有具体的问题或需求，比如注册、查询或删除用户数据，请明确告知，我将为您指引正确的流程。
Assistant: 看起来您可能正在经历一些情绪困扰。如果您愿意分享更多具体的问题或需要帮助的地方，我很乐意为您提供支持。无论是注册、数据查询还是删除用户信息，我都可以协助您处理。请告诉我您需要什么样的帮助？
Exiting conversation.


CWD: d:\ZXW2025\Code\LLM_learning\Agent_rumen
env_path: d:\ZXW2025\Code\LLM_learning\Agent_rumen\.env
loaded: True
HAS KEY: True
VALUE: 'sk-6a28680f8c7e4174b41ce71e235de4c8'
